In [13]:
import pandas as pd
import pingouin as pg
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, kendalltau
from read_data import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Main experiment

In [14]:
allsubjdata = get_main_exp_data()

In [15]:
for col in allsubjdata.columns:
    print(col)

scene
expected
initview
finalview
img_2
img_3
img_4
probe_1
probe_2
int_diff
samediff
task
simpleseq
zoomseq
p_exp
cond_practice
response
corr_resp
hit
rt
subject
date
time


In [ ]:
allsubjdata = allsubjdata.drop(columns=['samediff', 'task', 'simpleseq', 'zoomseq', 'cond_practice'])

In [ ]:
allsubjdata.to_csv('allsubjdata_clean.csv')

In [ ]:
allsubjdata.subject.nunique()

In [ ]:
allsubjdata.corr_resp.unique()

In [ ]:
allsubjdata.p_exp.unique()

In [ ]:
exclude_subjs = make_staircase_plots(allsubjdata)

In [ ]:
len(exclude_subjs)

In [ ]:
allsubjdata_noexcl = allsubjdata.copy()
allsubjdata = allsubjdata[~allsubjdata.subject.isin(exclude_subjs)]

In [ ]:
allsubjdata.subject.nunique()

In [ ]:
allsubjdata = compute_dprimes(allsubjdata)
allsubjdata_noexcl = compute_dprimes(allsubjdata_noexcl)

In [ ]:
allsubjdata.response.value_counts(dropna=False)

In [ ]:
p75data = allsubjdata[allsubjdata['p_exp']==0.75]
p50data = allsubjdata[allsubjdata['p_exp']==0.5]
p25data = allsubjdata[allsubjdata['p_exp']==0.25]

In [ ]:
p75data[(p75data['subject']==8)&(p75data['expected']==1)].dprime.unique()

In [ ]:
p75data[(p75data['subject']==8)&(p75data['expected']==0)].dprime.unique()

In [ ]:
p75data.subject.nunique()

In [ ]:
p75means = p75data.groupby(['subject', 'expected']).mean().reset_index()
p50means = p50data.groupby(['subject', 'expected']).mean().reset_index()
p25means = p25data.groupby(['subject', 'expected']).mean().reset_index()
allmeans = allsubjdata.groupby(['subject', 'expected', 'p_exp']).mean().reset_index()

In [ ]:
allmeans['hit'].corr(allmeans['criterion'], method='pearson')

In [ ]:
allmeans['hit'].corr(allmeans['dprime'], method='pearson')

In [ ]:
allmeans[allmeans['expected']==0]['dprime'].corr(allmeans['criterion'], method='pearson')

In [ ]:
allmeans[allmeans['expected']==1]['dprime'].corr(allmeans['criterion'], method='pearson')

In [ ]:
overallmeans = allmeans.groupby(['subject']).mean().reset_index()

In [ ]:
# scatter plot of hit rate and d'
plot = sns.jointplot(x='hit', y='dprime', data=allmeans[allmeans['expected']==1], palette='tab10')
plot.ax_joint.axvline(0.5, linestyle='--', color='grey')
plot.ax_joint.axhline(0.0, linestyle='--', color='grey')
plot.ax_marg_x.axvline(0.5, linestyle='--', color='grey')
plot.ax_marg_y.axhline(0.0, linestyle='--', color='grey')

In [ ]:
# scatter plot of hit rate and d'
plot = sns.jointplot(x='hit', y='dprime', data=allmeans[allmeans['expected']==0], palette='tab10')
plot.ax_joint.axvline(0.5, linestyle='--', color='grey')
plot.ax_joint.axhline(0.0, linestyle='--', color='grey')
plot.ax_marg_x.axvline(0.5, linestyle='--', color='grey')
plot.ax_marg_y.axhline(0.0, linestyle='--', color='grey')

In [ ]:
# scatter plot of hit rate and criterion
plot = sns.jointplot(x='hit', y='criterion', data=allmeans[allmeans['expected']==0], 
                     hue='p_exp', palette='tab10')
plot.ax_joint.axvline(0.5, linestyle='--', color='grey')
plot.ax_joint.axhline(0.0, linestyle='--', color='grey')
plot.ax_marg_x.axvline(0.5, linestyle='--', color='grey')
plot.ax_marg_y.axhline(0.0, linestyle='--', color='grey')

In [ ]:
# scatter plot of hit rate and criterion
plot = sns.jointplot(x='hit', y='criterion', data=allmeans[allmeans['expected']==1], 
                     hue='p_exp', palette='tab10')
plot.ax_joint.axvline(0.5, linestyle='--', color='grey')
plot.ax_joint.axhline(0.0, linestyle='--', color='grey')
plot.ax_marg_x.axvline(0.5, linestyle='--', color='grey')
plot.ax_marg_y.axhline(0.0, linestyle='--', color='grey')

In [ ]:
# scatter plot of d' and criterion
plot = sns.jointplot(x='dprime', y='criterion', data=allmeans[allmeans['expected']==1], palette='tab10')
plot.ax_joint.axvline(0.0, linestyle='--', color='grey')
plot.ax_joint.axhline(0.0, linestyle='--', color='grey')
plot.ax_marg_x.axvline(0.0, linestyle='--', color='grey')
plot.ax_marg_y.axhline(0.0, linestyle='--', color='grey')

In [ ]:
# scatter plot of d' and criterion
plot = sns.jointplot(x='dprime', y='criterion', data=allmeans[allmeans['expected']==0], palette='tab10')
plot.ax_joint.axvline(0.0, linestyle='--', color='grey')
plot.ax_joint.axhline(0.0, linestyle='--', color='grey')
plot.ax_marg_x.axvline(0.0, linestyle='--', color='grey')
plot.ax_marg_y.axhline(0.0, linestyle='--', color='grey')

In [ ]:
allmeans[allmeans['expected']==0]['hit'].corr(allmeans['criterion'], method='pearson')

## Sanity checks

In [ ]:
allmeans.groupby(['subject', 'p_exp']).mean().reset_index().groupby(['p_exp'])[['hit', 'criterion']].agg(['mean', 'sem'])

In [ ]:
pg.ttest(p75means.groupby(['subject']).mean().reset_index()['criterion'], 0.0)

In [ ]:
pg.ttest(p50means.groupby(['subject']).mean().reset_index()['criterion'], 0.0)

In [ ]:
pg.ttest(p25means.groupby(['subject']).mean().reset_index()['criterion'], 0.0)

## Statistics

In [ ]:
p75means.groupby(['expected'])[['hit']].agg(['mean', 'sem'])

In [ ]:
pg.ttest(p75means[p75means['expected']==1].hit, p75means[p75means['expected']==0].hit, paired=True)

In [ ]:
p50means.groupby(['expected'])[['hit']].agg(['mean', 'sem'])

In [ ]:
pg.ttest(p50means[p50means['expected']==1].hit, p50means[p50means['expected']==0].hit, paired=True)

In [ ]:
p25means.groupby(['expected'])[['hit']].agg(['mean', 'sem'])

In [ ]:
pg.ttest(p25means[p25means['expected']==1].hit, p25means[p25means['expected']==0].hit, paired=True)

In [ ]:
pg.ttest(p75means[p75means['expected']==1].criterion, p75means[p75means['expected']==0].criterion, paired=True)

In [ ]:
pg.ttest(p75means[p75means['expected']==1].dprime, p75means[p75means['expected']==0].dprime, paired=True)

In [ ]:
p50means.groupby(['expected'])[['criterion', 'dprime']].agg(['mean', 'sem'])

In [ ]:
pg.ttest(p50means[p50means['expected']==1].criterion, p50means[p50means['expected']==0].criterion, paired=True)

In [ ]:
pg.ttest(p50means[p50means['expected']==1].hit, p50means[p50means['expected']==0].hit, paired=True)

In [ ]:
pg.ttest(p50means[p50means['expected']==1].dprime, p50means[p50means['expected']==0].dprime, paired=True)

In [ ]:
p25means.groupby(['expected'])[['criterion', 'dprime', 'hit']].agg(['mean', 'sem'])

In [ ]:
pg.ttest(p25means[p25means['expected']==1].hit, p25means[p25means['expected']==0].hit, paired=True)

In [ ]:
pg.ttest(p25means[p25means['expected']==1].criterion, p25means[p25means['expected']==0].criterion, paired=True)

In [ ]:
pg.ttest(p25means[p25means['expected']==1].dprime, p25means[p25means['expected']==0].dprime, paired=True)

In [ ]:
p75means.groupby(['expected'])[['hit', 'dprime', 'criterion']].agg(['mean', 'sem'])

In [ ]:
p50means.groupby(['expected'])[['hit', 'dprime', 'criterion']].agg(['mean', 'sem'])

In [ ]:
p25means.groupby(['expected'])[['hit', 'dprime', 'criterion']].agg(['mean', 'sem'])

In [ ]:
allmeans_samesample = allmeans[~((allmeans['p_exp']==0.25)&(allmeans['subject'].isin([10, 130])))]

In [ ]:
allmeans_samesample[allmeans_samesample['p_exp']==0.25].subject.nunique()

In [ ]:
aov = pg.mixed_anova(dv='hit', within='expected',
                             between='p_exp',
                             subject='subject', 
                             data=allmeans)
pg.print_table(aov)

In [ ]:
aov

In [ ]:
allmeans.expected.unique()

In [ ]:
allhitdiffs = []
allpexp = []
for pe in allmeans.p_exp.unique():
    thesediffs = list(allmeans[(allmeans['p_exp']==pe)&(allmeans['expected']==1)].hit.values - \
        allmeans[(allmeans['p_exp']==pe)&(allmeans['expected']==0)].hit.values)
    allhitdiffs.extend(thesediffs)
    allpexp.extend([pe] * len(thesediffs))
allhitdiffs = pd.DataFrame(list(zip(allpexp, allhitdiffs)), columns =['p_exp', 'diff'])
    

In [ ]:
pairwisepvals = []
for pair in [[0.75, 0.5], [0.75, 0.25], [0.5, 0.25]]:
    tstats = pg.ttest(allhitdiffs[allhitdiffs['p_exp']==pair[0]]['diff'], allhitdiffs[allhitdiffs['p_exp']==pair[1]]['diff'])
    print(f"{pair[0]} vs. {pair[1]}")
    pg.print_table(tstats)
    pairwisepvals.append(tstats['p-val'])

In [ ]:
pg.multicomp(pairwisepvals, method='bonf')

In [ ]:
#thisdiff.shape

In [ ]:
pairwise_stats = pg.pairwise_ttests(dv='hit', 
                                    between='p_exp', 
                                    within='expected', 
                                    subject='subject',
                                    data=allmeans)
pg.print_table(pairwise_stats)

In [ ]:
aov = pg.mixed_anova(dv='dprime', within='expected',
                             between='p_exp',
                             subject='subject', 
                             data=allmeans)
pg.print_table(aov)

In [ ]:
aov = pg.mixed_anova(dv='criterion', within='expected',
                             between='p_exp',
                             subject='subject', 
                             data=allmeans)
pg.print_table(aov)

In [ ]:
sns.barplot(x='p_exp', y='dprime', hue='expected', 
            data=allmeans, order=[0.75, 0.5, 0.25], hue_order=[1, 0])

In [ ]:
sns.barplot(x='p_exp', y='criterion', hue='expected', 
             data=allmeans, order=[0.75, 0.5, 0.25], hue_order=[1, 0])

## Pretty plots

In [ ]:
make_pretty_plot(p75means, measure='hit')

In [ ]:
make_pretty_plot(p75means, measure='dprime')

In [ ]:
make_pretty_plot(p75means, measure='criterion')

In [ ]:
make_pretty_plot(p50means, measure='criterion')

In [ ]:
make_pretty_plot(p50means, measure='hit')

In [ ]:
make_pretty_plot(p50means, measure='dprime')

In [ ]:
make_pretty_plot(p25means, measure='hit')

In [ ]:
make_pretty_plot(p25means, measure='criterion')

In [ ]:
make_pretty_plot(p25means, measure='dprime')

## Results with no exclusions

In [ ]:
p75means_noexcl = allsubjdata_noexcl[allsubjdata_noexcl['p_exp']==0.75].groupby(
    ['subject', 'expected']).mean().reset_index()
p50means_noexcl = allsubjdata_noexcl[allsubjdata_noexcl['p_exp']==0.5].groupby(
    ['subject', 'expected']).mean().reset_index()
p25means_noexcl = allsubjdata_noexcl[allsubjdata_noexcl['p_exp']==0.25].groupby(
    ['subject', 'expected']).mean().reset_index()
allmeans_noexcl = allsubjdata_noexcl.groupby([
    'subject', 'expected', 'p_exp']).mean().reset_index()

### Summary statistics

In [ ]:
p75means_noexcl.groupby(['expected'])[['hit']].agg(['mean', 'sem'])

In [ ]:
p50means_noexcl.groupby(['expected'])[['hit']].agg(['mean', 'sem'])

In [ ]:
p25means_noexcl.groupby(['expected'])[['hit']].agg(['mean', 'sem'])

### Statistical tests

In [ ]:
pg.ttest(p75means_noexcl[p75means_noexcl['expected']==1].hit, p75means_noexcl[p75means_noexcl['expected']==0].hit, paired=True)

In [ ]:
pg.ttest(p50means_noexcl[p50means_noexcl['expected']==1].hit, p50means_noexcl[p50means_noexcl['expected']==0].hit, paired=True)

In [ ]:
pg.ttest(p25means_noexcl[p25means_noexcl['expected']==1].hit, p25means_noexcl[p25means_noexcl['expected']==0].hit, paired=True)

### Plots

In [ ]:
make_pretty_plot(p75means_noexcl, measure='hit', excl=False, fname='rot_p75_hit_noexcl.pdf')

In [ ]:
make_pretty_plot(p50means_noexcl, measure='hit', excl=False, fname='rot_p50_hit_noexcl.pdf')

In [ ]:
make_pretty_plot(p25means_noexcl, measure='hit', excl=False, fname='rot_p25_hit_noexcl.pdf')

## Aligned vs. Misaligned

In [ ]:
almisaldata = allsubjdata.copy()
almisaldata['aligned']=allsubjdata['initview'].isin([0, 180])
for al in almisaldata.aligned.unique():
    almisaldata[almisaldata['aligned']==al] = compute_dprimes(almisaldata[almisaldata['aligned']==al])
almisalmeans = almisaldata.groupby(['p_exp', 'aligned', 'expected', 'subject']).mean().reset_index()

In [ ]:
almisalmeans.head()

In [ ]:
almisalmeans.groupby(['aligned']).hit.mean()

In [ ]:
almisalmeans.groupby(['expected']).hit.mean()

In [ ]:
aov = pg.rm_anova(almisalmeans, dv='hit', within=['aligned', 'expected'], subject='subject')
pg.print_table(aov)

In [ ]:
aov = pg.rm_anova(almisalmeans[almisalmeans['p_exp'].isin([0.75])], dv='hit', within=['aligned', 'expected'], subject='subject')
pg.print_table(aov)

In [ ]:
aov = pg.rm_anova(almisalmeans[almisalmeans['p_exp'].isin([0.5])], dv='hit', within=['aligned', 'expected'], subject='subject')
pg.print_table(aov)

In [ ]:
aov = pg.rm_anova(almisalmeans[almisalmeans['p_exp'].isin([0.25])], dv='hit', within=['aligned', 'expected'], subject='subject')
pg.print_table(aov)

In [ ]:
almisaldata['p_exp'].unique()

In [ ]:
almisaldata[(almisaldata['aligned']==True)&(almisaldata['expected']==1)].dprime.mean()

In [ ]:
almisaldata[(almisaldata['aligned']==True)&(almisaldata['expected']==0)].dprime.mean()

In [ ]:
almisaldata[(almisaldata['aligned']==False)&(almisaldata['expected']==1)].dprime.mean()

In [ ]:
almisaldata[(almisaldata['aligned']==False)&(almisaldata['expected']==0)].dprime.mean()

### Aligned vs. Misaligned plot

In [ ]:
almisalmeans.head()

In [ ]:
alldiffs_acc = make_align_plot(almisalmeans, measure='hit', split=False)

In [ ]:
pal = list(sns.color_palette('Set2'))[:2]

In [ ]:
pal

In [ ]:
list(sns.color_palette('Paired'))[0]

In [ ]:
pal = [[list(sns.color_palette('Paired'))[0], list(sns.color_palette('Paired'))[6]],
       [list(sns.color_palette('Paired'))[1], list(sns.color_palette('Paired'))[7]]]

In [ ]:
sns.color_palette(pal[0])

In [ ]:
np.mean(np.array([[0.6509803921568628, 0.807843137254902, 0.8901960784313725],
                  [0.12156862745098039, 0.47058823529411764, 0.7058823529411765]]), axis=0)

In [ ]:
np.mean(np.array([[0.9921568627450981, 0.7490196078431373, 0.43529411764705883],
                  [1.0, 0.4980392156862745, 0.0]]), axis=0)

In [ ]:
sns.color_palette([(0.38627451, 0.63921569, 0.79803922)])

In [ ]:
sns.color_palette([(0.99607843, 0.62352941, 0.21764706)])

In [ ]:
pal

In [ ]:
make_align_plot_together(almisalmeans, measure='hit', split=False)

In [ ]:
make_align_plot_together(almisalmeans, measure='hit', split=False)

In [ ]:
make_align_plot_together(almisalmeans, measure='hit', split=False)

In [ ]:
almisalmeans_avg = almisalmeans.groupby(['subject', 'expected', 'aligned']).mean().reset_index()

In [ ]:
almisalmeans_avg.to_csv("almisalmeans.csv")

In [ ]:
len(almisalmeans_avg)

In [ ]:
pg.ttest(almisalmeans[(almisalmeans['aligned']==True)&(almisalmeans['expected']==1)]['hit'],
         almisalmeans[(almisalmeans['aligned']==True)&(almisalmeans['expected']==0)]['hit'], paired=True)

In [ ]:
pg.ttest(almisalmeans[(almisalmeans['aligned']==False)&(almisalmeans['expected']==1)]['hit'],
         almisalmeans[(almisalmeans['aligned']==False)&(almisalmeans['expected']==0)]['hit'], paired=True)

In [ ]:
alldiffs_acc.groupby(['aligned']).difference.mean()

In [ ]:
aov = pg.mixed_anova(data=alldiffs_acc, within='aligned', between='p_exp', dv='difference', subject='subject')
pg.print_table(aov)

In [ ]:
alldiffs_acc.head()

In [ ]:
acc_pvals = []
for pe in [0.75, 0.5, 0.25]:
    for al in [True, False]:
        print('p(exp) =', pe, 'aligned:', al)
        t_table = pg.ttest(alldiffs_acc[(alldiffs_acc['p_exp']==pe)&(alldiffs_acc['aligned']==al)]['difference'], 0.0)
        pg.print_table(t_table)
    print('Contrast:')
    t_table = pg.ttest(alldiffs_acc[(alldiffs_acc['p_exp']==pe)&(alldiffs_acc['aligned']==True)]['difference'],
                      alldiffs_acc[(alldiffs_acc['p_exp']==pe)&(alldiffs_acc['aligned']==False)]['difference'],
                      paired=True)
    acc_pvals.append(t_table['p-val'][0])
    pg.print_table(t_table)
    this_n = t_table['dof'][0] + 1
    bf = pg.bayesfactor_ttest(t_table['T'][0], this_n, paired=True)
    print('BF01:', 1/float(bf), '\n')
    

In [ ]:
pg.ttest(alldiffs_acc[alldiffs_acc['aligned']==True]['difference'],
         alldiffs_acc[alldiffs_acc['aligned']==False]['difference'],
                      paired=True)

In [ ]:
1/bf

In [ ]:
acc_pvals

In [ ]:
pg.multicomp(acc_pvals, method='bonf')

In [ ]:
alldiffs_crit = make_align_plot(almisalmeans, measure='criterion')

In [ ]:
aov = pg.rm_anova(dv='hit', within=['aligned', 'expected'],
                  subject='subject', data=almisalmeans[almisalmeans['p_exp']==0.25])
pg.print_table(aov)

In [ ]:
aov = pg.mixed_anova(dv='difference', within='aligned', between='p_exp', 
                  subject='subject',
                  data=alldiffs_acc)
pg.print_table(aov)

In [ ]:
aov = pg.mixed_anova(dv='difference', within='aligned', between='p_exp', 
                  subject='subject',
                  data=alldiffs_crit)
pg.print_table(aov)

In [ ]:
diffs_pvals = []

In [ ]:
stats = pg.ttest(alldiffs_dpr[(alldiffs_dpr['p_exp']==0.75)&(alldiffs_dpr['aligned']==True)].difference,
         alldiffs_dpr[(alldiffs_dpr['p_exp']==0.75)&(alldiffs_dpr['aligned']==False)].difference, paired=True)
diffs_pvals.append(stats['p-val'][0])
stats

In [ ]:
stats = pg.ttest(alldiffs_dpr[(alldiffs_dpr['p_exp']==0.5)&(alldiffs_dpr['aligned']==True)].difference,
                 alldiffs_dpr[(alldiffs_dpr['p_exp']==0.5)&(alldiffs_dpr['aligned']==False)].difference, paired=True)
diffs_pvals.append(stats['p-val'][0])
stats

In [ ]:
stats = pg.ttest(alldiffs_dpr[(alldiffs_dpr['p_exp']==0.25)&(alldiffs_dpr['aligned']==True)].difference,
                 alldiffs_dpr[(alldiffs_dpr['p_exp']==0.25)&(alldiffs_dpr['aligned']==False)].difference, paired=True)
diffs_pvals.append(stats['p-val'][0])
stats

In [ ]:
pg.multicomp(diffs_pvals, method='holm')

In [ ]:
diffs_pvals = []

In [ ]:
stats = pg.ttest(alldiffs_crit[(alldiffs_crit['p_exp']==0.75)&(alldiffs_crit['aligned']==True)].difference,
         alldiffs_crit[(alldiffs_crit['p_exp']==0.75)&(alldiffs_crit['aligned']==False)].difference, paired=True)
diffs_pvals.append(stats['p-val'][0])
stats

In [ ]:
stats = pg.ttest(alldiffs_crit[(alldiffs_crit['p_exp']==0.5)&(alldiffs_crit['aligned']==True)].difference,
                 alldiffs_crit[(alldiffs_crit['p_exp']==0.5)&(alldiffs_crit['aligned']==False)].difference, paired=True)
diffs_pvals.append(stats['p-val'][0])
stats

In [ ]:
stats = pg.ttest(alldiffs_crit[(alldiffs_crit['p_exp']==0.25)&(alldiffs_crit['aligned']==True)].difference,
                 alldiffs_crit[(alldiffs_crit['p_exp']==0.25)&(alldiffs_crit['aligned']==False)].difference, paired=True)
diffs_pvals.append(stats['p-val'][0])
stats

In [ ]:
pg.multicomp(diffs_pvals, method='bonf')

## Aligned vs. Misaligned - Exp. 1

In [ ]:
pvals_exp1 = []

In [ ]:
stats = pg.ttest(almisalmeans[(almisalmeans['p_exp']==0.75)&(almisalmeans['aligned']==True)&(almisalmeans['expected']==1)].dprime,
                 almisalmeans[(almisalmeans['p_exp']==0.75)&(almisalmeans['aligned']==True)&(almisalmeans['expected']==0)].dprime, paired=True)
pvals_exp1.append(stats['p-val'][0])
stats

In [ ]:
stats = pg.ttest(almisalmeans[(almisalmeans['p_exp']==0.75)&(almisalmeans['aligned']==False)&(almisalmeans['expected']==1)].dprime,
                 almisalmeans[(almisalmeans['p_exp']==0.75)&(almisalmeans['aligned']==False)&(almisalmeans['expected']==0)].dprime, paired=True)
pvals_exp1.append(stats['p-val'][0])
stats

In [ ]:
stats = pg.ttest(almisalmeans[(almisalmeans['p_exp']==0.75)&(almisalmeans['aligned']==True)&(almisalmeans['expected']==1)].criterion,
                 almisalmeans[(almisalmeans['p_exp']==0.75)&(almisalmeans['aligned']==True)&(almisalmeans['expected']==0)].criterion, paired=True)
pvals_exp1.append(stats['p-val'][0])
stats

In [ ]:
pvals_exp1

In [ ]:
pg.ttest(almisalmeans[(almisalmeans['p_exp']==0.75)&(almisalmeans['aligned']==False)&(almisalmeans['expected']==1)].criterion,
         almisalmeans[(almisalmeans['p_exp']==0.75)&(almisalmeans['aligned']==False)&(almisalmeans['expected']==0)].criterion, paired=True)

In [ ]:
aov = pg.rm_anova(dv='dprime', within=['expected', 'aligned'], 
                  subject='subject',
                  data=almisalmeans[almisalmeans['p_exp']==0.75])
pg.print_table(aov)

In [ ]:
aov = pg.rm_anova(dv='criterion', within=['expected', 'aligned'], 
                  subject='subject',
                  data=almisalmeans[almisalmeans['p_exp']==0.75])
pg.print_table(aov)

## Aligned vs. Misaligned - Exp. 2

In [ ]:
pg.ttest(almisalmeans[(almisalmeans['p_exp']==0.5)&(almisalmeans['aligned']==True)&(almisalmeans['expected']==1)].dprime,
         almisalmeans[(almisalmeans['p_exp']==0.5)&(almisalmeans['aligned']==True)&(almisalmeans['expected']==0)].dprime, paired=True)

In [ ]:
pg.ttest(almisalmeans[(almisalmeans['p_exp']==0.5)&(almisalmeans['aligned']==False)&(almisalmeans['expected']==1)].dprime,
         almisalmeans[(almisalmeans['p_exp']==0.5)&(almisalmeans['aligned']==False)&(almisalmeans['expected']==0)].dprime, paired=True)

In [ ]:
pg.ttest(almisalmeans[(almisalmeans['p_exp']==0.5)&(almisalmeans['aligned']==True)&(almisalmeans['expected']==1)].criterion,
         almisalmeans[(almisalmeans['p_exp']==0.5)&(almisalmeans['aligned']==True)&(almisalmeans['expected']==0)].criterion, paired=True)

In [ ]:
pg.ttest(almisalmeans[(almisalmeans['p_exp']==0.5)&(almisalmeans['aligned']==False)&(almisalmeans['expected']==1)].criterion,
         almisalmeans[(almisalmeans['p_exp']==0.5)&(almisalmeans['aligned']==False)&(almisalmeans['expected']==0)].criterion, paired=True)

In [ ]:
aov = pg.rm_anova(dv='dprime', within=['expected', 'aligned'], 
                  subject='subject',
                  data=almisalmeans[almisalmeans['p_exp']==0.5])
pg.print_table(aov)

In [ ]:
aov = pg.rm_anova(dv='criterion', within=['expected', 'aligned'], 
                  subject='subject',
                  data=almisalmeans[almisalmeans['p_exp']==0.5])
pg.print_table(aov)

In [ ]:
aov['p-unc'][2]

In [ ]:
sns.barplot(x='aligned', y='criterion', hue='expected', 
             data=almisalmeans[almisalmeans['p_exp']==0.25], hue_order=[1, 0])

In [ ]:
sns.barplot(x='aligned', y='dprime', hue='expected', 
             data=almisalmeans[almisalmeans['p_exp']==0.25], hue_order=[1, 0])

### Across time

In [ ]:
rot_dprimetime = dprime_across_time(allsubjdata)

In [ ]:
fig, ax1 = plt.subplots()
plot = sns.lineplot(x='winno', y='dprime', 
                    data=rot_dprimetime[(rot_dprimetime['p_exp']==0.75)&(rot_dprimetime['expected']==1)],
                    color='red')
plot.set(xticklabels=[])
plot.set(xlabel=None)
ax2 = ax1.twiny()
plot = sns.lineplot(x='winno', y='dprime',
                   data=rot_dprimetime[(rot_dprimetime['p_exp']==0.75)&(rot_dprimetime['expected']==0)],
                   color='blue')
plot.set(xticklabels=[])
plot.set(xlabel=None)

In [ ]:
fig, ax1 = plt.subplots()
plot = sns.lineplot(x='winno', y='criterion', 
                    data=rot_dprimetime[(rot_dprimetime['p_exp']==0.75)&(rot_dprimetime['expected']==1)],
                    color='red')
plot.set(xticklabels=[])
plot.set(xlabel=None)
ax2 = ax1.twiny()
plot = sns.lineplot(x='winno', y='criterion',
                   data=rot_dprimetime[(rot_dprimetime['p_exp']==0.75)&(rot_dprimetime['expected']==0)],
                   color='blue')
plot.set(xticklabels=[])
plot.set(xlabel=None)

In [ ]:
fig, ax1 = plt.subplots()
plot = sns.lineplot(x='winno', y='dprime', 
                    data=rot_dprimetime[(rot_dprimetime['p_exp']==0.5)&(rot_dprimetime['expected']==1)],
                    color='red')
plot.set(xticklabels=[])
plot.set(xlabel=None)
ax2 = ax1.twiny()
plot = sns.lineplot(x='winno', y='dprime',
                   data=rot_dprimetime[(rot_dprimetime['p_exp']==0.5)&(rot_dprimetime['expected']==0)],
                   color='blue')
plot.set(xticklabels=[])
plot.set(xlabel=None)

In [ ]:
fig, ax1 = plt.subplots()
plot = sns.lineplot(x='winno', y='criterion', 
                    data=rot_dprimetime[(rot_dprimetime['p_exp']==0.5)&(rot_dprimetime['expected']==1)],
                    color='red')
plot.set(xticklabels=[])
plot.set(xlabel=None)
ax2 = ax1.twiny()
plot = sns.lineplot(x='winno', y='criterion',
                   data=rot_dprimetime[(rot_dprimetime['p_exp']==0.5)&(rot_dprimetime['expected']==0)],
                   color='blue')
plot.set(xticklabels=[])
plot.set(xlabel=None)

In [ ]:
fig, ax1 = plt.subplots()
plot = sns.lineplot(x='winno', y='dprime', 
                    data=rot_dprimetime[(rot_dprimetime['p_exp']==0.25)&(rot_dprimetime['expected']==1)],
                    color='red')
plot.set(xticklabels=[])
plot.set(xlabel=None)
ax2 = ax1.twiny()
plot = sns.lineplot(x='winno', y='dprime',
                   data=rot_dprimetime[(rot_dprimetime['p_exp']==0.25)&(rot_dprimetime['expected']==0)],
                   color='blue')
plot.set(xticklabels=[])
plot.set(xlabel=None)

In [ ]:
fig, ax1 = plt.subplots()
plot = sns.lineplot(x='winno', y='criterion', 
                    data=rot_dprimetime[(rot_dprimetime['p_exp']==0.25)&(rot_dprimetime['expected']==1)],
                    color='red')
plot.set(xticklabels=[])
plot.set(xlabel=None)
ax2 = ax1.twiny()
plot = sns.lineplot(x='winno', y='criterion',
                   data=rot_dprimetime[(rot_dprimetime['p_exp']==0.25)&(rot_dprimetime['expected']==0)],
                   color='blue')
plot.set(xticklabels=[])
plot.set(xlabel=None)

## Sanity checks and survey

In [16]:
allsanchecks, allsurveys = get_sanchecks_and_surveys(allsubjdata)

In [19]:
allsurveys = allsurveys.drop(columns=['samediff', 'task', 'simpleseq', 'zoomseq', 'cond_practice'])
allsanchecks = allsanchecks.drop(columns=['samediff', 'task', 'simpleseq', 'zoomseq', 'cond_practice'])

In [20]:
allsurveys.head()

,sequence_attention,object_expect,percent_expected,comments,p_exp,subject,date,time
2,4.0,3.0,65,I tried to unfocus my eyes a little as that se...,0.5,3,20210831,1630419047809
3,2.0,3.0,50,I honestly got confused originally. I thought ...,0.5,4,20210901,1630509449935
4,4.0,5.0,40,NaN,0.5,5,20210901,1630508551387
5,1.0,3.0,50,NaN,0.5,6,20210901,1630522365396
6,1.0,4.0,70,The experiment was too tiring and eye strainin...,0.5,7,20210902,1630569991648


In [21]:
allsurveys.subject.nunique()

271

In [22]:
allsurveys.to_csv('allsurveys_clean.csv')

In [ ]:
allsanchecks.to_csv('allsanchecks_clean.csv')

In [ ]:
allsanchecks.head()

In [18]:
for col in allsurveys.columns:
    print(col)

sequence_attention
object_expect
percent_expected
comments
samediff
task
simpleseq
zoomseq
p_exp
cond_practice
subject
date
time


In [ ]:
allsurveys.sort_values(['p_exp'],ascending=False).groupby(['p_exp'])[['sequence_attention']].agg(['mean', 'sem'])

In [ ]:
allsurveys.sort_values(['p_exp'],ascending=False).groupby(['p_exp'])[['object_expect']].agg(['mean', 'sem'])

In [ ]:
allsurveys.sort_values(['p_exp'],ascending=False).groupby(['p_exp'])[['percent_expected']].agg(['mean', 'sem'])

In [ ]:
aov = pg.welch_anova(dv='sequence_attention', between='p_exp', data=allsurveys)

pg.print_table(aov)

In [ ]:
aov = pg.welch_anova(dv='object_expect', between='p_exp', data=allsurveys)

pg.print_table(aov)

In [ ]:
aov = pg.welch_anova(dv='percent_expected', between='p_exp', data=allsurveys)

pg.print_table(aov)

In [ ]:
allmeans.head()

In [ ]:
print('==================')
print('Overall')
print('------------------')
accdiffs = allmeans[allmeans['expected']==1].hit.values - \
           allmeans[allmeans['expected']==0].hit.values
critdiffs = allmeans[allmeans['expected']==1].criterion.values - \
            allmeans[allmeans['expected']==0].criterion.values
dprdiffs = allmeans[allmeans['expected']==1].dprime.values - \
           allmeans[allmeans['expected']==0].dprime.values
print('Accuracy-sequence attention correlation:')
stats = pg.corr(allsurveys.sequence_attention, accdiffs, method='pearson')
print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')
# print('Criterion-sequence attention correlation:')
# stats = pg.corr(allsurveys.sequence_attention, critdiffs, method='pearson')
# print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')
# print('DPrime-sequence attention correlation:')
# stats = pg.corr(allsurveys.sequence_attention, dprdiffs, method='pearson')
# print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')
#tau, p = kendalltau(allsurveys[allsurveys['p_exp']==pe].sequence_attention, critdiffs, nan_policy='omit')
#print('tau =', tau, '- p =', p, '\n')
print('Accuracy-object prediction correlation:')
stats = pg.corr(allsurveys.object_expect, accdiffs, method='pearson')
print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')
# print('Criterion-object prediction correlation:')
# stats = pg.corr(allsurveys.object_expect, critdiffs, method='pearson')
# print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')
# print('DPrime-object prediction correlation:')
# stats = pg.corr(allsurveys.object_expect, dprdiffs, method='pearson')
# print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')
#tau, p = kendalltau(allsurveys[allsurveys['p_exp']==pe].object_expect, critdiffs, nan_policy='omit')
#print('tau =', tau, '- p =', p, '\n')
print('Accuracy-probability estimate correlation:')
stats = pg.corr(allsurveys.percent_expected, accdiffs, method='pearson')
print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')
# print('Criterion-probability estimate correlation:')
# stats = pg.corr(allsurveys.percent_expected, critdiffs, method='pearson')
# print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')
# print('DPrime-probability estimate correlation:')
# stats = pg.corr(allsurveys.percent_expected, dprdiffs, method='pearson')
# print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')

for pe in [0.75, 0.5, 0.25]:
    print('==================')
    print('P(Exp) =', pe)
    print('------------------')
    thispexp = allmeans[allmeans['p_exp']==pe]
    critdiffs = thispexp[thispexp['expected']==1].criterion.values - \
                thispexp[thispexp['expected']==0].criterion.values
    dprdiffs = thispexp[thispexp['expected']==1].dprime.values - \
               thispexp[thispexp['expected']==0].dprime.values
    accdiffs = thispexp[thispexp['expected']==1].hit.values - \
               thispexp[thispexp['expected']==0].hit.values
    print('Accuracy-sequence attention correlation:')
    stats = pg.corr(allsurveys[allsurveys['p_exp']==pe].sequence_attention, accdiffs, method='pearson')
    print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')
    # print('Criterion-sequence attention correlation:')
    # stats = pg.corr(allsurveys[allsurveys['p_exp']==pe].sequence_attention, critdiffs, method='pearson')
    # print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')
    # print('DPrime-sequence attention correlation:')
    # stats = pg.corr(allsurveys[allsurveys['p_exp']==pe].sequence_attention, dprdiffs, method='pearson')
    # print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')
    #tau, p = kendalltau(allsurveys[allsurveys['p_exp']==pe].sequence_attention, critdiffs, nan_policy='omit')
    #print('tau =', tau, '- p =', p, '\n')
    print('-------------------------------')
    print('Accuracy-object prediction correlation:')
    stats = pg.corr(allsurveys[allsurveys['p_exp']==pe].object_expect, accdiffs, method='pearson')
    print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')
    # print('Criterion-object prediction correlation:')
    # stats = pg.corr(allsurveys[allsurveys['p_exp']==pe].object_expect, critdiffs, method='pearson')
    # print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')
    # print('DPrime-object prediction correlation:')
    # stats = pg.corr(allsurveys[allsurveys['p_exp']==pe].object_expect, dprdiffs, method='pearson')
    # print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')
    #tau, p = kendalltau(allsurveys[allsurveys['p_exp']==pe].object_expect, critdiffs, nan_policy='omit')
    #print('tau =', tau, '- p =', p, '\n')
    print('-------------------------------')
    print('Accuracy-probability estimate correlation:')
    stats = pg.corr(allsurveys[allsurveys['p_exp']==pe].percent_expected, accdiffs, method='pearson')
    print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')
    # print('Criterion-probability estimate correlation:')
    # stats = pg.corr(allsurveys[allsurveys['p_exp']==pe].percent_expected, critdiffs, method='pearson')
    # print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')
    # print('DPrime-probability estimate correlation:')
    # stats = pg.corr(allsurveys[allsurveys['p_exp']==pe].percent_expected, dprdiffs, method='pearson')
    # print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')
    #tau, p = kendalltau(allsurveys[allsurveys['p_exp']==pe].percent_expected, critdiffs, nan_policy='omit')
    #print('tau =', tau, '- p =', p, '\n')
    
    #print('DPrime-sequence attention correlation:')
    #tau, p = kendalltau(allsurveys[allsurveys['p_exp']==pe].sequence_attention, dprdiffs, nan_policy='omit')
    #print('tau =', tau, 'p =', p, '\n')
    #print('DPrime-object prediction correlation:')
    #tau, p = kendalltau(allsurveys[allsurveys['p_exp']==pe].object_expect, dprdiffs, nan_policy='omit')
    #print('tau =', tau, 'p =', p, '\n')
    #print('DPrime-probability estimate correlation:')
    #tau, p = kendalltau(allsurveys[allsurveys['p_exp']==pe].percent_expected, dprdiffs, nan_policy='omit')
    #print('tau =', tau, 'p =', p, '\n')

## Participant demographics (rotation)

In [ ]:
# Participant demographics
alldemodata = pd.DataFrame()
subjdemo = glob('./prolific_export_*.csv')
renamedict = {'Participant id': 'participant_id', 'Age': 'age', 'Sex': 'sex'}
for sd in subjdemo:
    thisdemodata = pd.read_csv(sd, engine='python', error_bad_lines=False)
    thisdemodata = thisdemodata.rename(columns=renamedict)
    thisdemodata = thisdemodata[list(renamedict.values())]
    alldemodata = pd.concat([alldemodata, thisdemodata], ignore_index=True)

incl_ids = [i for i in allsanchecks.prolific_id]
alldemodata = alldemodata[alldemodata['participant_id'].isin(incl_ids)]    

demo_probs = []
for thisid in alldemodata.participant_id:
    thissub = allsanchecks[allsanchecks['prolific_id']==thisid]
    demo_probs.append(thissub.p_exp.values[0])
alldemodata['p_exp'] = demo_probs

In [ ]:
alldemodata[alldemodata['p_exp']==0.75].age.mean()

In [ ]:
alldemodata[alldemodata['p_exp']==0.25].sex.value_counts()

In [ ]:
thisdemodata.head()

In [ ]:
renamedict = {'Participant id': 'participant_id', 'Age': 'age', 'Sex': 'Sex'}

In [ ]:
alldemodata = alldemodata[list(renamedict.values())]

In [ ]:
list(renamedict.values())

In [ ]:
alldemodata.head()

In [ ]:
allsanchecks.head()

In [ ]:
len(incl_ids)

In [ ]:
incl_ids[0]

In [ ]:
alldemodata['Participant id'][100]

In [ ]:
'56cb572b73cd5b0006f79acd' in incl_ids

In [ ]:
incl_demogr = alldemodata[alldemodata['Participant id'].isin(incl_ids)]

In [ ]:
probs = []
for demrow in incl_demogr.iterrows():
    print(demrow[1]['Participant id'])
    #thisid = demrow['Participant id'][0]
    #print(thisid)
    #probs.append(allsanchecks[allsanchecks])

In [ ]:
incl_demogr.status.unique() # should be all 'APPROVED'

In [ ]:
incl_demogr['Sex'].value_counts()

In [ ]:
incl_demogr['age'].mean()

In [ ]:
incl_demogr['age'].std()

# Zoom experiment

In [ ]:
zoomdata = get_zoom_exp_data()

In [ ]:
zoomdata.p_exp.unique()

In [ ]:
zoomdata.date.unique()

In [ ]:
zoomdata = zoomdata[(zoomdata.date>=20220511)&
                    (zoomdata.time>=1652000000000)&
                    (zoomdata.cond_practice==False)]

In [ ]:
zoomdata.subject.nunique()

In [ ]:
exclude_subjs = make_staircase_plots(zoomdata)

In [ ]:
len(exclude_subjs)

In [ ]:
# How many missing subjects per experiment
for pe in [0.75, 0.5, 0.25]:
    print(pe, zoomdata[(zoomdata['p_exp']==pe)&(zoomdata.subject.isin(exclude_subjs))].subject.nunique())

In [ ]:
zoomdata = zoomdata[~zoomdata.subject.isin(exclude_subjs)]

In [ ]:
zoomdata

In [ ]:
zoomdata.subject.nunique()

In [ ]:
zoomdata.response.value_counts(dropna=False)

In [ ]:
zoomdata.corr_resp.value_counts()

#
## Compute d' and criterion

In [ ]:
zoomdata = compute_dprimes(zoomdata)

#

In [ ]:
trialnos = []
for i, s in enumerate(zoomdata.subject.unique()):
    thissub = zoomdata[zoomdata['subject']==s]
    thissub = thissub[96:]
    if i==0:
        zoomdata_cut = thissub
    else:
        zoomdata_cut = pd.concat([zoomdata_cut, thissub])

In [ ]:
len(zoomdata_cut)/151

In [ ]:
zoomdata_cut['int_diff'] = zoomdata_cut['int_diff'].apply(np.abs)

In [ ]:
zoomcutmeans = zoomdata_cut.groupby(['p_exp', 'subject']).mean().reset_index()

In [ ]:
len(zoomcutmeans)

In [ ]:
zoomcutmeans.groupby(['p_exp'])[['int_diff']].agg(['mean', 'std'])

In [ ]:
zoomdata.groupby(['p_exp', 'expected'])[['hit', 'dprime', 'criterion']].agg(['mean', 'sem'])

In [ ]:
p50means_cut = zoomdata_cut[zoomdata_cut['p_exp']==0.5].groupby(['subject', 'expected']).mean().reset_index()
pg.ttest(p50means_cut[p50means_cut['expected']==1]['criterion'], 
         p50means_cut[p50means_cut['expected']==0]['criterion'], paired=True)

In [ ]:
p25means_cut = zoomdata_cut[zoomdata_cut['p_exp']==0.25].groupby(['subject', 'expected']).mean().reset_index()
pg.ttest(p25means_cut[p25means_cut['expected']==1]['criterion'], 
         p25means_cut[p25means_cut['expected']==0]['criterion'], paired=True)

In [ ]:
zoomp75means = zoomdata[zoomdata['p_exp']==0.75].groupby(['subject', 'expected']).mean().reset_index()
zoomp50means = zoomdata[zoomdata['p_exp']==0.5].groupby(['subject', 'expected']).mean().reset_index()
zoomp25means = zoomdata[zoomdata['p_exp']==0.25].groupby(['subject', 'expected']).mean().reset_index()
allzoommeans = zoomdata.groupby(['subject', 'expected', 'p_exp']).mean().reset_index()

In [ ]:
pg.ttest(zoomp75means.groupby(['subject']).mean().reset_index()['criterion'], 0.0)

In [ ]:
pg.ttest(zoomp50means.groupby(['subject']).mean().reset_index()['criterion'], 0.0)

In [ ]:
pg.ttest(zoomp25means.groupby(['subject']).mean().reset_index()['criterion'], 0.0)

In [ ]:
zoomp50means.groupby(['subject']).mean().reset_index()['criterion'].mean()

In [ ]:
allzoommeans.groupby(['subject', 'p_exp']).mean().reset_index().groupby(['p_exp'])[['hit', 'criterion']].agg(['mean', 'sem'])

In [ ]:
# scatter plot of hit rate and criterion
plot = sns.jointplot(x='hit', y='dprime', data=allzoommeans[allzoommeans['expected']==1], 
                     hue='p_exp', palette='tab10')
plot.ax_joint.axvline(0.5, linestyle='--', color='grey')
plot.ax_joint.axhline(0.0, linestyle='--', color='grey')
plot.ax_marg_x.axvline(0.5, linestyle='--', color='grey')
plot.ax_marg_y.axhline(0.0, linestyle='--', color='grey')

In [ ]:
# scatter plot of hit rate and criterion
plot = sns.jointplot(x='hit', y='criterion', data=allzoommeans[allzoommeans['expected']==1], 
                     hue='p_exp', palette='tab10')
plot.ax_joint.axvline(0.5, linestyle='--', color='grey')
plot.ax_joint.axhline(0.0, linestyle='--', color='grey')
plot.ax_marg_x.axvline(0.5, linestyle='--', color='grey')
plot.ax_marg_y.axhline(0.0, linestyle='--', color='grey')

In [ ]:
zoomp75means.groupby(['expected'])[['criterion', 'dprime']].agg(['mean', 'sem'])

In [ ]:
pg.ttest(zoomp75means[zoomp75means['expected']==1]['criterion'], 
         zoomp75means[zoomp75means['expected']==0]['criterion'], paired=True)

In [ ]:
pg.ttest(zoomp75means[zoomp75means['expected']==1]['dprime'], 
         zoomp75means[zoomp75means['expected']==0]['dprime'], paired=True)

In [ ]:
zoomp50means.groupby(['expected'])[['criterion', 'dprime']].agg(['mean', 'sem'])

In [ ]:
pg.ttest(zoomp50means[zoomp50means['expected']==1]['criterion'], 
         zoomp50means[zoomp50means['expected']==0]['criterion'], paired=True)

In [ ]:
pg.ttest(zoomp50means[zoomp50means['expected']==1]['dprime'], 
         zoomp50means[zoomp50means['expected']==0]['dprime'], paired=True)

In [ ]:
pg.ttest(zoomp25means[zoomp25means['expected']==1]['criterion'], 
         zoomp25means[zoomp25means['expected']==0]['criterion'], paired=True)

In [ ]:
pg.ttest(zoomp25means[zoomp25means['expected']==1]['dprime'], 
         zoomp25means[zoomp25means['expected']==0]['dprime'], paired=True)

In [ ]:
zoomp50means['hit'].corr(zoomp50means['criterion'], method='pearson')

In [ ]:
p50means = zoomdata[zoomdata['p_exp']==0.5].groupby(['subject', 'expected']).mean().reset_index()
pg.ttest(p50means[p50means['expected']==1]['criterion'], p50means[p50means['expected']==0]['criterion'], paired=True)

In [ ]:
p75means = zoomdata[zoomdata['p_exp']==0.75].groupby(['subject', 'expected']).mean().reset_index()
pg.ttest(p75means[p75means['expected']==1]['dprime'], p75means[p75means['expected']==0]['dprime'], paired=True)

In [ ]:
zoomp75means['hit'].corr(zoomp75means['criterion'], method='pearson')

In [ ]:
zoomp50means['hit'].corr(zoomp50means['criterion'], method='pearson')

In [ ]:
zoomp25means['hit'].corr(zoomp25means['criterion'], method='pearson')

In [ ]:
pg.ttest(allzoommeans[(allzoommeans['p_exp']==0.75)&(allzoommeans['expected']==0)]['dprime'], 
         allzoommeans[(allzoommeans['p_exp']==0.5)&(allzoommeans['expected']==0)]['dprime'], paired=False)

In [ ]:
sns.barplot(x='p_exp', y='dprime', hue='expected', 
             data=allzoommeans, order=[0.75, 0.5, 0.25], hue_order=[1, 0])

In [ ]:
_ = sns.barplot(x='p_exp', y='dprime', hue='expected', 
            data=allzoommeans, order=[0.75, 0.5, 0.25], hue_order=[1, 0])

In [ ]:
make_pretty_plot(zoomp75means, measure='dprime', rotzoom='zoom')

In [ ]:
aov = pg.mixed_anova(dv='dprime', within='expected',
                             between='p_exp',
                             subject='subject', 
                             data=zoomdata[zoomdata.p_exp.isin([0.75, 0.5, 0.25])])
pg.print_table(aov)

In [ ]:
aov = pg.mixed_anova(dv='criterion', within='expected',
                             between='p_exp',
                             subject='subject', 
                             data=zoomdata[zoomdata.p_exp.isin([0.75, 0.5, 0.25])])
pg.print_table(aov)

In [ ]:
aov = pg.mixed_anova(dv='hit', within='expected',
                             between='p_exp',
                             subject='subject', 
                             data=zoomdata[zoomdata.p_exp.isin([0.75, 0.5, 0.25])])
pg.print_table(aov)

In [ ]:
zoomdata[zoomdata['p_exp']==0.25].subject.unique()

### Survey data

In [ ]:
zoomsanchecks, zoomsurveys = get_sanchecks_and_surveys(zoomdata)

In [ ]:
zoomsurveys.head()

In [ ]:
zoomsurveys.sort_values(['p_exp'],ascending=False).groupby(['p_exp'])[['sequence_attention', 'object_expect', 'percent_expected']].agg(['mean', 'sem'])

In [ ]:
aov = pg.welch_anova(dv='sequence_attention', between='p_exp', data=zoomsurveys)

pg.print_table(aov)

In [ ]:
aov = pg.welch_anova(dv='object_expect', between='p_exp', data=zoomsurveys)

pg.print_table(aov)

In [ ]:
aov = pg.welch_anova(dv='percent_expected', between='p_exp', data=zoomsurveys)

pg.print_table(aov)

In [ ]:
allzoommeans.head()

In [ ]:
# Use deviation from the correct probability instead of absolute value?

#zoomsurveys['percent_expected'] = np.abs(zoomsurveys['percent_expected'].values - 100*zoomsurveys['p_exp'].values)

In [ ]:
print('==================')
print('Overall')
print('------------------')
critdiffs = allzoommeans[allzoommeans['expected']==1].criterion.values - \
            allzoommeans[allzoommeans['expected']==0].criterion.values
dprdiffs = allzoommeans[allzoommeans['expected']==1].dprime.values - \
           allzoommeans[allzoommeans['expected']==0].dprime.values
print('Criterion-sequence attention correlation:')
stats = pg.corr(zoomsurveys.sequence_attention, critdiffs, method='pearson')
print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')
print('DPrime-sequence attention correlation:')
stats = pg.corr(zoomsurveys.sequence_attention, dprdiffs, method='pearson')
print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')
#tau, p = kendalltau(zoomsurveys[zoomsurveys['p_exp']==pe].sequence_attention, critdiffs, nan_policy='omit')
#print('tau =', tau, '- p =', p, '\n')
print('Criterion-object prediction correlation:')
stats = pg.corr(zoomsurveys.object_expect, critdiffs, method='pearson')
print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')
print('DPrime-object prediction correlation:')
stats = pg.corr(zoomsurveys.object_expect, dprdiffs, method='pearson')
print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')
#tau, p = kendalltau(zoomsurveys[zoomsurveys['p_exp']==pe].object_expect, critdiffs, nan_policy='omit')
#print('tau =', tau, '- p =', p, '\n')
print('Criterion-probability estimate correlation:')
stats = pg.corr(zoomsurveys.percent_expected, critdiffs, method='pearson')
print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')
print('DPrime-probability estimate correlation:')
stats = pg.corr(zoomsurveys.percent_expected, dprdiffs, method='pearson')
print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')

for pe in [0.75, 0.5, 0.25]:
    print('==================')
    print('P(Exp) =', pe)
    print('------------------')
    thispexp = allzoommeans[allzoommeans['p_exp']==pe]
    critdiffs = thispexp[thispexp['expected']==1].criterion.values - \
                thispexp[thispexp['expected']==0].criterion.values
    dprdiffs = thispexp[thispexp['expected']==1].dprime.values - \
               thispexp[thispexp['expected']==0].dprime.values
    print('Criterion-sequence attention correlation:')
    stats = pg.corr(zoomsurveys[zoomsurveys['p_exp']==pe].sequence_attention, critdiffs, method='pearson')
    print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')
    print('DPrime-sequence attention correlation:')
    stats = pg.corr(zoomsurveys[zoomsurveys['p_exp']==pe].sequence_attention, dprdiffs, method='pearson')
    print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')
    #tau, p = kendalltau(zoomsurveys[zoomsurveys['p_exp']==pe].sequence_attention, critdiffs, nan_policy='omit')
    #print('tau =', tau, '- p =', p, '\n')
    print('Criterion-object prediction correlation:')
    stats = pg.corr(zoomsurveys[zoomsurveys['p_exp']==pe].object_expect, critdiffs, method='pearson')
    print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')
    print('DPrime-object prediction correlation:')
    stats = pg.corr(zoomsurveys[zoomsurveys['p_exp']==pe].object_expect, dprdiffs, method='pearson')
    print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')
    #tau, p = kendalltau(zoomsurveys[zoomsurveys['p_exp']==pe].object_expect, critdiffs, nan_policy='omit')
    #print('tau =', tau, '- p =', p, '\n')
    print('Criterion-probability estimate correlation:')
    stats = pg.corr(zoomsurveys[zoomsurveys['p_exp']==pe].percent_expected, critdiffs, method='pearson')
    print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')
    print('DPrime-probability estimate correlation:')
    stats = pg.corr(zoomsurveys[zoomsurveys['p_exp']==pe].percent_expected, dprdiffs, method='pearson')
    print('r =', stats['r'][0], '- p =', stats['p-val'][0], '- BF01 =', 1/float(stats['BF10'][0]), '\n')
    #tau, p = kendalltau(zoomsurveys[zoomsurveys['p_exp']==pe].percent_expected, critdiffs, nan_policy='omit')
    #print('tau =', tau, '- p =', p, '\n')
    
    #print('DPrime-sequence attention correlation:')
    #tau, p = kendalltau(zoomsurveys[zoomsurveys['p_exp']==pe].sequence_attention, dprdiffs, nan_policy='omit')
    #print('tau =', tau, 'p =', p, '\n')
    #print('DPrime-object prediction correlation:')
    #tau, p = kendalltau(zoomsurveys[zoomsurveys['p_exp']==pe].object_expect, dprdiffs, nan_policy='omit')
    #print('tau =', tau, 'p =', p, '\n')
    #print('DPrime-probability estimate correlation:')
    #tau, p = kendalltau(zoomsurveys[zoomsurveys['p_exp']==pe].percent_expected, dprdiffs, nan_policy='omit')
    #print('tau =', tau, 'p =', p, '\n')

In [ ]:
sceneattend_data = pd.DataFrame({
    '0.75': [len(zoomsurveys[(zoomsurveys['p_exp']==0.75)&(zoomsurveys['sequence_attention']==i)]) for i in range(7)],
    '0.50': [len(zoomsurveys[(zoomsurveys['p_exp']==0.50)&(zoomsurveys['sequence_attention']==i)]) for i in range(7)],
    '0.25': [len(zoomsurveys[(zoomsurveys['p_exp']==0.25)&(zoomsurveys['sequence_attention']==i)]) for i in range(7)]
}, index=[1,2,3,4,5,6,7])
sceneattend_data = sceneattend_data.apply(lambda x: x/sum(x)*100, axis=0)

In [ ]:
ax = sceneattend_data.transpose().plot(kind='bar', stacked=True, colormap='tab10', edgecolor='none')
plt.xticks(rotation=0)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['top'].set_visible(False)
#ax.spines['bottom'].set_visible(False)
ax.tick_params(axis=u'both', which=u'both',length=0)
ax.legend(bbox_to_anchor=(1.1, 1.05), fancybox=False, framealpha=1, edgecolor='black')
plt.savefig('plots/zoom_sceneattend.pdf')

In [ ]:
zoomsurveys.object_expect.unique()

In [ ]:
objexpect_data = pd.DataFrame({
    '0.75': [len(zoomsurveys[(zoomsurveys['p_exp']==0.75)&(zoomsurveys['object_expect']==i)]) for i in range(7)],
    '0.50': [len(zoomsurveys[(zoomsurveys['p_exp']==0.50)&(zoomsurveys['object_expect']==i)]) for i in range(7)],
    '0.25': [len(zoomsurveys[(zoomsurveys['p_exp']==0.25)&(zoomsurveys['object_expect']==i)]) for i in range(7)]
}, index=[1,2,3,4,5,6,7])
objexpect_data = objexpect_data.apply(lambda x: x/sum(x)*100, axis=0)

In [ ]:
objexpect_data

In [ ]:
ax = objexpect_data.transpose().plot(kind='bar', stacked=True, colormap='tab10', edgecolor='none')
plt.xticks(rotation=0)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['top'].set_visible(False)
#ax.spines['bottom'].set_visible(False)
ax.tick_params(axis=u'both', which=u'both',length=0)
ax.legend(bbox_to_anchor=(1.1, 1.05), fancybox=False, framealpha=1, edgecolor='black')
plt.savefig('plots/zoom_objexpect.pdf')

In [ ]:
zoomsurveys[zoomsurveys['p_exp']==0.25]

In [ ]:
for com in zoomsurveys[zoomsurveys['p_exp']==0.25].comments:
    print(com)
    print('----------')

### Across time

In [ ]:
zoomtseries = get_measures_in_time(zoomdata)

In [ ]:
dprimetime = dprime_across_time(zoomdata)

In [ ]:
dprimetime.head()

In [ ]:
fig, ax1 = plt.subplots()
plot = sns.lineplot(x='winno', y='dprime', 
                    data=dprimetime[(dprimetime['p_exp']==0.75)&(dprimetime['expected']==1)],
                    color='red')
#plot.set(ylim=(0., 1.2))
plot.set(xticklabels=[])
plot.set(xlabel=None)
ax2 = ax1.twiny()
plot = sns.lineplot(x='winno', y='dprime',
                   data=dprimetime[(dprimetime['p_exp']==0.75)&(dprimetime['expected']==0)],
                   color='blue')
#plot.set(ylim=(0., 1.2))
plot.set(xticklabels=[])
plot.set(xlabel=None)

In [ ]:
fig, ax1 = plt.subplots()
plot = sns.lineplot(x='winno', y='criterion', 
                    data=dprimetime[(dprimetime['p_exp']==0.75)&(dprimetime['expected']==1)],
                    color='red')
#plot.set(ylim=(0., 1.2))
plot.set(xticklabels=[])
plot.set(xlabel=None)
ax2 = ax1.twiny()
plot = sns.lineplot(x='winno', y='criterion',
                   data=dprimetime[(dprimetime['p_exp']==0.75)&(dprimetime['expected']==0)],
                   color='blue')
#plot.set(ylim=(0., 1.2))
plot.set(xticklabels=[])
plot.set(xlabel=None)

In [ ]:
fig, ax1 = plt.subplots()
plot = sns.lineplot(x='winno', y='dprime', 
                    data=dprimetime[(dprimetime['p_exp']==0.5)&(dprimetime['expected']==1)],
                    color='red')
#plot.set(ylim=(0., 1.2))
plot.set(xticklabels=[])
plot.set(xlabel=None)
ax2 = ax1.twiny()
plot = sns.lineplot(x='winno', y='dprime',
                   data=dprimetime[(dprimetime['p_exp']==0.5)&(dprimetime['expected']==0)],
                   color='blue')
#plot.set(ylim=(0., 1.2))
plot.set(xticklabels=[])
plot.set(xlabel=None)

In [ ]:
fig, ax1 = plt.subplots()
plot = sns.lineplot(x='winno', y='criterion', 
                    data=dprimetime[(dprimetime['p_exp']==0.5)&(dprimetime['expected']==1)],
                    color='red')
#plot.set(ylim=(0., 1.2))
plot.set(xticklabels=[])
plot.set(xlabel=None)
ax2 = ax1.twiny()
plot = sns.lineplot(x='winno', y='criterion',
                   data=dprimetime[(dprimetime['p_exp']==0.5)&(dprimetime['expected']==0)],
                   color='blue')
#plot.set(ylim=(0., 1.2))
plot.set(xticklabels=[])
plot.set(xlabel=None)

In [ ]:
fig, ax1 = plt.subplots()
plot = sns.lineplot(x='winno', y='dprime', 
                    data=dprimetime[(dprimetime['p_exp']==0.25)&(dprimetime['expected']==1)],
                    color='red')
#plot.set(ylim=(0., 1.2))
plot.set(xticklabels=[])
plot.set(xlabel=None)
ax2 = ax1.twiny()
plot = sns.lineplot(x='winno', y='dprime',
                   data=dprimetime[(dprimetime['p_exp']==0.25)&(dprimetime['expected']==0)],
                   color='blue')
#plot.set(ylim=(0., 1.2))
plot.set(xticklabels=[])
plot.set(xlabel=None)

In [ ]:
fig, ax1 = plt.subplots()
plot = sns.lineplot(x='winno', y='criterion', 
                    data=dprimetime[(dprimetime['p_exp']==0.25)&(dprimetime['expected']==1)],
                    color='red')
#plot.set(ylim=(0., 1.2))
plot.set(xticklabels=[])
plot.set(xlabel=None)
ax2 = ax1.twiny()
plot = sns.lineplot(x='winno', y='criterion',
                   data=dprimetime[(dprimetime['p_exp']==0.25)&(dprimetime['expected']==0)],
                   color='blue')
#plot.set(ylim=(0., 1.2))
plot.set(xticklabels=[])
plot.set(xlabel=None)

In [ ]:
len(dprimetime)

In [ ]:
zoomdata.p_exp.nunique()==3

In [ ]:
zoomtseries.head()

In [ ]:
fig, ax1 = plt.subplots()
plot = sns.lineplot(x='trialno', y='hit', 
                    data=zoomtseries[(zoomtseries['p_exp']==0.75)&(zoomtseries['expected']==1)],
                    color='red')
plot.set(ylim=(0.5, 0.9))
plot.set(xticklabels=[])
plot.set(xlabel=None)
ax2 = ax1.twiny()
plot = sns.lineplot(x='trialno', y='hit',
                   data=zoomtseries[(zoomtseries['p_exp']==0.75)&(zoomtseries['expected']==0)],
                   color='blue')
plot.set(ylim=(0.5, 0.9))
plot.set(xticklabels=[])
plot.set(xlabel=None)

In [ ]:
fig, ax1 = plt.subplots()
plot = sns.lineplot(x='trialno', y='hit', 
                    data=zoomtseries[(zoomtseries['p_exp']==0.5)&(zoomtseries['expected']==1)],
                    color='red')
plot.set(ylim=(0.5, 0.9))
plot.set(xticklabels=[])
plot.set(xlabel=None)
ax2 = ax1.twiny()
plot = sns.lineplot(x='trialno', y='hit',
                   data=zoomtseries[(zoomtseries['p_exp']==0.5)&(zoomtseries['expected']==0)],
                   color='blue')
plot.set(ylim=(0.5, 0.9))
plot.set(xticklabels=[])
plot.set(xlabel=None)

In [ ]:
fig, ax1 = plt.subplots()
plot = sns.lineplot(x='trialno', y='hit', 
                    data=zoomtseries[(zoomtseries['p_exp']==0.25)&(zoomtseries['expected']==1)],
                    color='red')
plot.set(ylim=(0.5, 0.9))
plot.set(xticklabels=[])
plot.set(xlabel=None)
ax2 = ax1.twiny()
plot = sns.lineplot(x='trialno', y='hit',
                   data=zoomtseries[(zoomtseries['p_exp']==0.25)&(zoomtseries['expected']==0)],
                   color='blue')
plot.set(ylim=(0.5, 0.9))
plot.set(xticklabels=[])
plot.set(xlabel=None)

## Group experiments together

In [ ]:
allsubjdata.head()

In [ ]:
allsubjdata['condition'] = 'rotation'

In [ ]:
zoomdata.head()

In [ ]:
zoomdata['condition'] = 'zoom'

In [ ]:
allexpsdata = pd.concat([allsubjdata, zoomdata])

In [ ]:
allexpsdata

In [ ]:
allexpsdata.subject.nunique()

In [ ]:
# Get differences
allexpsmeandiffs = []
for s in allexpsdata.subject.unique():
    thissub = allexpsdata[allexpsdata['subject']==s]
    assert((thissub.p_exp.nunique()==1)&(thissub.condition.nunique()==1))
    thisdiff_acc = thissub[thissub['expected']==1].hit.mean() - thissub[thissub['expected']==0].hit.mean()
    thisdiff_dpr = thissub[thissub['expected']==1].dprime.mean() - thissub[thissub['expected']==0].dprime.mean()
    thisdiff_crt = thissub[thissub['expected']==1].criterion.mean() - thissub[thissub['expected']==0].criterion.mean()
    thisrow = {'subject': s, 'condition': thissub.condition.iloc[0], 'p_exp': thissub.p_exp.iloc[0],
               'diff_acc': thisdiff_acc, 'diff_dprime': thisdiff_dpr, 'diff_crit': thisdiff_crt}
    allexpsmeandiffs.append(thisrow)
allexpsmeandiffs = pd.DataFrame(allexpsmeandiffs)

In [ ]:
allexpsmeandiffs

In [ ]:
aov = pg.anova(dv='diff_dprime', between=['condition', 'p_exp'],
                             data=allexpsmeandiffs[allexpsmeandiffs['p_exp'].isin([0.5, 0.25])])
pg.print_table(aov)

In [ ]:
aov = pg.anova(dv='diff_crit', between=['condition', 'p_exp'],
                             data=allexpsmeandiffs[allexpsmeandiffs['p_exp'].isin([0.75, 0.5])])
pg.print_table(aov)

## Participant demographics (zoom)

In [ ]:
# Participant demographics
zoomdemodata = pd.DataFrame()
subjdemo = glob('./prolific_export_*.csv')
renamedict = {'Participant id': 'participant_id', 'Age': 'age', 'Sex': 'sex'}
for sd in subjdemo:
    thisdemodata = pd.read_csv(sd, engine='python', error_bad_lines=False)
    thisdemodata = thisdemodata.rename(columns=renamedict)
    thisdemodata = thisdemodata[list(renamedict.values())]
    zoomdemodata = pd.concat([zoomdemodata, thisdemodata], ignore_index=True)

In [ ]:
for pid in zoomsanchecks[zoomsanchecks['p_exp']==0.75].prolific_id:
    if pid not in list(zoomdemodata.participant_id):
        print(pid)

In [ ]:
incl_ids = [i.strip() for i in zoomsanchecks.prolific_id]
zoomdemodata = zoomdemodata[zoomdemodata['participant_id'].isin(incl_ids)]    

demo_probs = []
for thisid in zoomdemodata.participant_id:
    thissub = zoomsanchecks[zoomsanchecks['prolific_id']==thisid]
    demo_probs.append(thissub.p_exp.values[0])
zoomdemodata['p_exp'] = demo_probs

In [ ]:
len(zoomdemodata[zoomdemodata['p_exp']==0.75])

In [ ]:
zoomdemodata[zoomdemodata['p_exp']==0.75].sex.value_counts()

In [ ]:
zoomdemodata[zoomdemodata['p_exp']==0.5].sex.value_counts()

In [ ]:
zoomdemodata[zoomdemodata['p_exp']==0.25].sex.value_counts()

In [ ]:
zoomdemodata.loc[zoomdemodata['age']=='DATA_EXPIRED', 'age'] = None
zoomdemodata.age = zoomdemodata.age.astype('float')

In [ ]:
zoomdemodata.age.unique()

In [ ]:
zoomdemodatagroupby(['p_exp'])[['age']].agg(['mean', 'std'])

In [ ]:
len(zoomdemodata[zoomdemodata['p_exp']==0.5])

#

In [ ]:
zoomdata.subject.nunique()

In [ ]:
zoomdata[zoomdata.date>=20220830].subject.nunique()

In [ ]:
zoomdata[zoomdata.date<20220819].subject.nunique()

In [ ]:
rand_subset = random.sample(list(zoomdata[zoomdata.date<20220819].subject.unique()), 5)

In [ ]:
rand_subset

In [ ]:
zoomdata = zoomdata[~zoomdata.subject.isin(rand_subset)]

In [ ]:
zoomdata[zoomdata.date<20220830].date.unique()

In [ ]:
zoomdata[zoomdata.date==20220819].subject.nunique()

In [ ]:
# compare with and without thorough explanation
expldata = zoomdata[(zoomdata.date<20220819)|(zoomdata.date>=20220830)]
expldata['training'] = np.nan

In [ ]:
expldata.subject.nunique()

In [ ]:
expldata.loc[expldata.date<20220819, 'training'] = 'easy'
expldata.loc[expldata.date>=20220830, 'training'] = 'hard'

In [ ]:
expldata.training.unique()

In [ ]:
expldata = compute_dprimes(expldata)

In [ ]:
expldata.groupby(['training', 'expected'])[['dprime']].agg(['mean', 'sem'])

In [ ]:
aov = pg.mixed_anova(dv='hit', within='expected',
                             between='training',
                             subject='subject', 
                             data=expldata)
pg.print_table(aov)

In [ ]:
aov = pg.mixed_anova(dv='dprime', within='expected',
                             between='training',
                             subject='subject', 
                             data=expldata)
pg.print_table(aov)

In [ ]:
sns.set(style='white')
sns.set(rc={'figure.figsize':(8, 5)})
b = sns.barplot(data=expldata, x='training', y='dprime', 
                hue_order = [1.0, 0.0],
                hue='expected', palette='colorblind', ci=95)
#b.set(ylim=(0.5, 1.0), ylabel='Accuracy')
plt.tight_layout()

In [ ]:
explmeans = expldata.groupby(['subject', 'training', 'expected']).mean().reset_index()

In [ ]:
hardmeans = explmeans[explmeans['training']=='hard']
pg.ttest(hardmeans[hardmeans['expected']==1]['dprime'],
         hardmeans[hardmeans['expected']==0]['dprime'], paired=True)

In [ ]:
easymeans = explmeans[explmeans['training']=='easy']
pg.ttest(easymeans[easymeans['expected']==1]['dprime'],
         easymeans[easymeans['expected']==0]['dprime'], paired=True)